# Used cars on German eBay

For this project, we will be working with some data that was scraped and uploaded to Kaggle. There are 50,000 data points we will be cleaning and analyzing. There are quite a few Series in this data set, so naturally there will be much to clean!

To get started, let's import our libraries:

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#autos = pd.read_csv('autos.csv')

You will notice that I've commented out the `pd.read_csv()`. When attempting to load the file, I encountered an encoding error. It seems that UTF-8, the most common encoding method for these filetypes (and what pandas uses as a default argument for the encoding parameter), was unsuccessful. Let's try a different encoding method and see if it is a success.

In [2]:
autos = pd.read_csv('autos.csv',encoding='Latin-1')

In [3]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


*Success!* This particular csv file was using a `Latin-1` encoding method. Using this argument, we were able to load in our Dataframe.

In [4]:
autos.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

## What's the info?
Taking a look at the list, we can immediately see a few issues. There are columns with null data. Specifically, the *vehicleType*, *gearbox*, *model*, *fuelType*, and *notRepairedDamage* columns. Below, I created a variable to isolate the null values in the *gearbox column*.

In [5]:
a_gear_missing = autos[autos['gearbox'].isnull()]

In [6]:
#df of NaN values in the gearbox column
a_gear_missing['vehicleType'].value_counts()

kleinwagen    403
limousine     306
kombi         233
bus           138
cabrio        110
coupe          62
suv            53
andere         32
Name: vehicleType, dtype: int64

## The headers

The column headers are currently in camelCase, which we don't want to work with. Instead, I'd like to convert these all to snake_case! Manually editing each line is one way of doing it, but that'd take too much time. And besides, what's programming good for if we can't do it the lazy way? ;)

Let's start by creating a function that uses a regex to convert the camelCase to snake_case and return a nicely formatted list that we will then assign to our `autos.columns`!

In [7]:
columns = autos.columns

In [8]:
def snake_me(col):
    a = re.compile('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))')
    new_list = []
    for i in col:
        i = a.sub(r'_\1',i).lower()
        new_list.append(i)
    return new_list
        
        
    

new_col = snake_me(columns)
new_col
        

['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'abtest',
 'vehicle_type',
 'year_of_registration',
 'gearbox',
 'power_ps',
 'model',
 'odometer',
 'month_of_registration',
 'fuel_type',
 'brand',
 'not_repaired_damage',
 'date_created',
 'nr_of_pictures',
 'postal_code',
 'last_seen']

Very nice! There are a few edits that I'd like to make to some of the column names, however. Now, we *could* make individual edits using `str.replace()` in a bunch of separate calls, but again... ***WHY?***. Let's use a nice fancy list comprehension instead!

In [9]:
new_col = [n.replace('year_of_registration','registration_year').replace('month_of_registration','registration_month').replace('not_repaired_damage','unrepaired_damage').replace('date_created','ad_created') for n in new_col]
    

In [10]:
autos.columns = new_col

In [11]:
autos.head(2)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08


## And just like that!

We've replaced the string values we wanted and have them all nicely formatted. Looking back, I should have replaced the values first and then ran it through the function to format the camelCase to snake_case, but what's learning if you don't make any mistakes? Taking a look at the column headers, we can see that our goal has been met!

## Let's explore our data next, shall we?

We've done some preliminary work, but the fun has just begun! What we want to do next is clean up some columns. Let's use the `.describe()` method to gain some insights. We will use the `include='all'` argument to gain both categorical and numeric columns.

In [12]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 15:49:30,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Taking a look, there are a few things that pop out. One, the *odometer* column can be adjusted by removing the 'km' and converting the entire Series to numeric data in order to perform aggregations on them. The *price* colum is also not numeric data as there are dollar signs galore. Let's fix this, shall we?

In [13]:
odom = autos['odometer']

In [14]:
odom = [o.replace('km','').replace('k','').replace(',','') for o in odom]

In [15]:
autos['odometer'] = odom
autos['odometer'] = autos['odometer'].astype(int)

In [16]:
autos.rename(columns={'odometer':'odometer_km'},inplace=True)
autos['odometer_km']

0        150000
1        150000
2         70000
3         70000
4        150000
          ...  
49995    100000
49996    150000
49997      5000
49998     40000
49999    150000
Name: odometer_km, Length: 50000, dtype: int64

## Odometer is now an int64 dtype!

And while we were at it, we removed any unwanted characters like *km* or *k*, and renamed the column from *odometer* to *odometer_km* for clarity. To avoid doing the whole `autos['odometer'] = autos.rename(columns={'odometer':'odometer_km'})`, we just used the `inplace=True` argument to push our changes to the DataFrame more succinctly.

### Now, getting back to the rest of the data set...

Let's explore what some columns entail. As we clean this data, we need to ask ourselves if there are any outliers, inconsistencies, or impossibilities within certain Series. We will explore our newly formatted ***odometer_km*** Series. Let's use a couple of attributes and methods to explore!

In [17]:
odom = autos['odometer_km']
odom_shape = odom.unique().shape
odom_desc = odom.describe()
odom_value = odom.value_counts().sort_index()

In [18]:
odom_shape[0]

13

In [19]:
odom_desc

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [20]:
odom_value

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

In [21]:
price = autos['price']

remove_dol = re.compile(r'[$,]')

stripped_price = []

for p in price:
    stripped = re.sub(remove_dol,"",p)
    p = float(stripped)
    stripped_price.append(p)
    
autos['price'] = stripped_price
autos.columns = (a.replace('price','price_dol') for a in autos.columns)


In [22]:
price = autos['price_dol'].value_counts()


In [23]:
autos.dtypes

date_crawled           object
name                   object
seller                 object
offer_type             object
price_dol             float64
abtest                 object
vehicle_type           object
registration_year       int64
gearbox                object
power_ps                int64
model                  object
odometer_km             int64
registration_month      int64
fuel_type              object
brand                  object
unrepaired_damage      object
ad_created             object
nr_of_pictures          int64
postal_code             int64
last_seen              object
dtype: object

In [24]:
autos['seller'].value_counts(normalize=True)

privat        0.99998
gewerblich    0.00002
Name: seller, dtype: float64